In [ ]:
# Import Dependencies
import numpy as np
import tensorflow as tf
from tensorflow import keras

from keras import backend as K

import os
from pathlib import Path

from keras.optimizers import Adam
from keras.models import Model
from keras.utils import load_img

import pprint

from google.colab import drive
drive.mount('/content/drive')

data_name = "Staten Island"

# Path
input_dir = ""
input_ext = ".png"
target_dir = ""
target_ext = ".png"

# Format
img_size = (256, 256)
input_band_count = 3    # only support 3 (RGB) for now

# Training HyperParameters
batch_size = 10
early_stop_patience = 5
epochs = 100
monitor_function = 'val_jaccard_coef_int' # Defined later

# Data Loader
class ImagesSequence(keras.utils.Sequence):
    """Helper to iterate over the data (as Numpy arrays)."""

    def __init__(self, batch_size, img_size, input_img_paths, target_img_paths):
        self.batch_size = batch_size
        self.img_size = img_size
        self.input_img_paths = input_img_paths
        self.target_img_paths = target_img_paths

    def __len__(self):  # no. of batches
        return len(self.target_img_paths) // self.batch_size

    def __getitem__(self, idx):
        """Returns tuple (input, target) correspond to batch #idx."""
        i = idx * self.batch_size
        batch_input_img_paths = self.input_img_paths[i : i + self.batch_size]
        batch_target_img_paths = self.target_img_paths[i : i + self.batch_size]
        x = np.zeros((self.batch_size,) + self.img_size + (input_band_count,), dtype="float32") #concatnation of tuples; (3,) stands for 1-element tuple

        for j, path in enumerate(batch_input_img_paths):
            if input_band_count == 3:
                img = load_img(path, target_size=self.img_size)
                x[j] = img
                x[j] = x[j] /  255 # Normalize
            else:
                raise NotImplementedError
        y = np.zeros((self.batch_size,) + self.img_size + (1,), dtype="uint8")
        for j, path in enumerate(batch_target_img_paths):
            img = load_img(path, target_size=self.img_size, color_mode="grayscale")
            y[j] = np.expand_dims(img, -1)/255  # Grayscale doesn't have last dimension
        return x, y

# Loadind and Spliting Data
# 10% for testing
# train:val = 4:1
input_img_paths = sorted(
    [
        os.path.join(input_dir, fname)
        for fname in os.listdir(input_dir)
        if fname.endswith(input_ext)
    ]
)
target_img_paths = sorted(
    [
        os.path.join(target_dir, fname)
        for fname in os.listdir(target_dir)
        if fname.endswith(target_ext) and not fname.startswith(".")
    ]
)

train_index = slice(None, int(0.72*len(os.listdir(input_dir))))
val_index = slice(int(0.72*len(os.listdir(input_dir))), int(0.9*len(os.listdir(input_dir))))
test_index = slice(int(0.9*len(os.listdir(input_dir))), None)

train_input_img_paths   = input_img_paths[train_index]
train_target_img_paths  = target_img_paths[train_index]
val_input_img_paths     = input_img_paths[val_index]
val_target_img_paths    = target_img_paths[val_index]

train_gen = ImagesSequence(batch_size, img_size, train_input_img_paths, train_target_img_paths)
val_gen = ImagesSequence(batch_size, img_size, val_input_img_paths, val_target_img_paths)

# Monitor Function
# Based on [Winning solution of SpaceNet2](https://github.com/SpaceNetChallenge/BuildingDetectors_Round2/tree/master/1-XD_XD)
# Minimize Jaccard distance := 1-Jaccard coeff, where coeff is intersection / union
def jaccard_coef_int(y_true, y_pred, smooth = 1e-12):
    y_pred_pos = K.round(K.clip(y_pred, 0, 1))
    intersection = K.sum(y_true * y_pred_pos, axis=[-1, -2, 0])
    sum_ = K.sum(y_true + y_pred_pos, axis=[-1, -2, 0])
    jac = (intersection + smooth) / (sum_ - intersection + smooth)
    return (1 - K.mean(jac)) * smooth

Mounted at /content/drive


In [ ]:
# Model HyperParameters
optimizer_name = "Adam"
learning_rate = 1e-4
weight_decay = 5e-5
loss='binary_crossentropy'
metrics=['accuracy', jaccard_coef_int]
activation = "GELU"
output_activation = "Sigmoid"

# Implementing Model
!pip3 install keras_unet_collection
from keras_unet_collection import models

model_name = "Unet"
optimizer = keras.optimizers.Adam(learning_rate=learning_rate, weight_decay=weight_decay)
model = models.unet_2d(input_size=(256,256,3), filter_num=[64, 128, 256, 512], n_labels=1, activation = activation, output_activation = output_activation)
model.compile(
    optimizer=optimizer,
    loss=loss,
    metrics=metrics
)

In [ ]:
model.summary()

In [ ]:
# Training and saving
progress_dir = Path(data_name+" Progress")
train_model_filename = Path(progress_dir, f"model_train.h5")
best_model_filename = Path(progress_dir, f"model_best.h5")
history_csv_filename = Path(progress_dir, f"loss_history.csv")
params_txt_filename = Path(progress_dir, f"params.txt")
model_txt_filename = Path(progress_dir, f"model_summary.txt")

def store_in_dict(*args):
    return {key: globals()[key] for key in args}

training_params = store_in_dict(
    "batch_size", "monitor_function", "early_stop_patience", "epochs"
)
model_params = store_in_dict(
    "optimizer_name", "learning_rate", "weight_decay", "loss", "metrics", "activation", "output_activation"
)

progress_dir.mkdir(parents=True, exist_ok=True)
with open(params_txt_filename, 'w') as f:
    f.write(pprint.pformat(store_in_dict(
        "data_name", "training_params", "model_name", "model_params"
    )))
with open(model_txt_filename, 'w') as f:
    model.summary(print_fn=lambda x: f.write(x + '\n'))


train_checkpoint_callback = keras.callbacks.ModelCheckpoint(str(train_model_filename), monitor=monitor_function, save_best_only=False)
best_checkpoint_callback = keras.callbacks.ModelCheckpoint(str(best_model_filename), monitor=monitor_function, save_best_only=True, mode="min", verbose=1)
logger_callback = keras.callbacks.CSVLogger(str(history_csv_filename), append=True)  # more fault-tolerant than output of model.fit()
earlystopping_callback = keras.callbacks.EarlyStopping(
    monitor=monitor_function,
    mode="min",
    patience=early_stop_patience,
    verbose=0)

callback_list = [
    train_checkpoint_callback,
    best_checkpoint_callback,
    logger_callback,
    earlystopping_callback
]

model.fit(train_gen, epochs=epochs, verbose=2, validation_data=val_gen, callbacks=callback_list)

Epoch 1/100


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(



Epoch 1: val_jaccard_coef_int improved from inf to 0.00000, saving model to Staten Island Progress/model_best.h5
427/427 - 4366s - loss: 0.3916 - accuracy: 0.8572 - jaccard_coef_int: 9.9353e-13 - val_loss: 0.4036 - val_accuracy: 0.8203 - val_jaccard_coef_int: 9.9893e-13 - 4366s/epoch - 10s/step
Epoch 2/100

Epoch 2: val_jaccard_coef_int improved from 0.00000 to 0.00000, saving model to Staten Island Progress/model_best.h5
427/427 - 56s - loss: 0.3140 - accuracy: 0.8695 - jaccard_coef_int: 8.5484e-13 - val_loss: 0.4430 - val_accuracy: 0.8247 - val_jaccard_coef_int: 9.6115e-13 - 56s/epoch - 130ms/step
Epoch 3/100

Epoch 3: val_jaccard_coef_int improved from 0.00000 to 0.00000, saving model to Staten Island Progress/model_best.h5
427/427 - 56s - loss: 0.2549 - accuracy: 0.8981 - jaccard_coef_int: 6.3154e-13 - val_loss: 0.3670 - val_accuracy: 0.8494 - val_jaccard_coef_int: 7.9688e-13 - 56s/epoch - 131ms/step
Epoch 4/100

Epoch 4: val_jaccard_coef_int improved from 0.00000 to 0.00000, savi